<table border="1" cellpadding="5" cellspacing="0" width="100%">
   <tr>
      <!--<td align="left" valign="top" width="120"><img src="./pictures/octopeek-logo.png" width="120" /></td>-->
      <td align="left" valign="top" width="120"><img src="https://media-exp1.licdn.com/dms/image/C510BAQE93sqc09g7qg/company-logo_200_200/0?e=2159024400&v=beta&t=Ebcbl-_mVoiGn-jo8xRX3V0iuAckEhZXnLKcmnOv2Wk" width="120" /></td>
      <td valign="top" width="100%" align="center"><h1><font color="blue">Enseignement d'Intégration - Sujet Octopeek</font></h1>
      <p align="right"><h2>Analyse de sentiments sur des Tweets</h2></p>
   </tr>
</table>

# Collecte de données Twitter

Ce notebook a pour objectif la collecte des données pour votre analyse

## Création d'une application Twitter

Pour accéder à l'API Twitter, vous devez créer un compte développeur :

<https://developer.twitter.com/en/apply-for-access>

Il sera associé à un compte Twitter (donc si vous n'en avez pas déjà un, il faudra le créer avant le compte développeur).

Puis vous créez une "application" (il faut chercher à partir de l'interface développeur). Vous devez entrer des informations concernant l'application afin que celle-ci soit validée par Twitter (c'est assez rapide en général). Expliquez (sur le formulaire ou en répondant au mail de Twitter) qu'elle est destinée à de l'analyse de données pour un projet étudiant sur l'analyse de sentiment. N'hésitez pas à bien détailler même si ce n'est pas tout à fait exacte. Vos réponses doivent être uniques afin que Twitter puisse valider en toute confiance votre application.

## Instanciation de l'API Twitter

In [1]:
# Installation de tweepy :
!pip install tweepy

In [2]:
# Ajout de vos identifiants et tokens (il y a un bouton qui vous permet de les générer sur le site web) :
consumer_key = 'MCYTnXGESGbqpekYJ6beDIQGf'
consumer_secret = '64sErRB2j6cDKKKItBD2kXHXLQrZVDVN6rlyfrvhM1HJdMwJdq'
access_token = '1269007715940130817-x3nGQqeqZHetNd9fVmKM6gHmGpK8Is'
access_token_secret = 'Y2BVlc6qidGffA0fG8RYca4JyvvL3OCufvp1QtoxcfaoV'


In [3]:
# En utilisant la documentation de Tweepy, importez `tweepy` et initialisez une instance de `tweepy.API` :
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


<div class="alert alert-block alert-success">
Vous trouverez toutes les informations nécessaires sur la <a href="http://docs.tweepy.org/en/latest/getting_started.html#hello-tweepy">documentation de Tweepy</a>
</div>

## Récolte des tweets

Récoltez au moins 20 000 tweets à partir de différentes requêtes que vous aurez établies en fonction des objectifs du projet. Utilisez [l'API search](http://docs.tweepy.org/en/latest/api.html#help-methods). Plus vous aurez de tweets, plus vos analyses seront fiables. Vous pouvez laisser tourner la récolte plusieurs heures.

Vous stockerez les tweets dans un dictionnaire avec comme clef les identifiants de tweets et comme valeur [l'objet Tweet](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object). N'oubliez pas de filtrer en langue anglaise. Utilisez un `Cursor` en cherchant dans la documentation Tweepy ou en trouvant des exemples. Vous aurez également besoin des [opérateurs](https://developer.twitter.com/en/docs/tweets/rules-and-filtering/overview/standard-operators) de recherche (e.g. les guillemets, le `OR`, l'opération `-`). Cherchez comment exclure les *retweets*. Certains tweets sont tronqués lorsqu'ils font plus de 180 caractères, vous pouvez utiliser la [version extended](https://github.com/tweepy/tweepy/issues/935).

Si vous avez créer plusieurs dictionnaire (pour chaque catégorie de requete par exemple), vous devez éliminer les doublons de tweets (par id).

In [4]:
# Votre code :
from datetime import datetime as dt
import tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
#days_from_now = [5] : permet de limiter la recherche date par date afin de ne charger partie par partie les tweets.


tweets = dict()


duplicates = 0 #initialisation du compteur de duplicatas
nb_items = 1000
cursor = tweepy.Cursor(api.search, "electric car -filter:retweets", lang='en', tweet_mode='extended')
if nb_items is None:
    items = cursor.items()
else:
    items = cursor.items(nb_items)
for result in items:
    tweet = result._json
    if 'full_text' in tweet:#Ici on retire le label full text et on remplace le texte par le full text.
        tweet['text'] = tweet['full_text']
        del tweet['full_text']
    current_id = tweet['id']
    if current_id in tweets:
        duplicates += 1
    tweets[current_id] = tweet
print("Already added tweets count in this loop: " + str(duplicates))
print("New tweets size: " + str(len(tweets)))


TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/search/tweets.json?q=electric+car+-filter%3Aretweets&max_id=1403246953585156096&lang=en&tweet_mode=extended (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1124)')))

<div class="alert alert-block alert-success">
Pensez à la fiabilité de vos données, si par exemple vous voulez les tweets parlant d'une décision politique de Trump vis-à-vis d'un sujet A (pour en avoir le sentiment général) mais que la majorité de vos tweets sont en réalité associé à un sujet connexe B, alors vos analyse seront faussées. Pour cela il faut regarder quelques exemples de Tweet (vous pouvez utiliser le site web Twitter et sa fonction de recherche).
</div>

<div class="alert alert-block alert-success">
Twitter impose une <a href="https://developer.twitter.com/en/docs/basics/rate-limiting">limite de requêtes sur une fenetre de temps</a>, donc si vous voulez accelerer votre récolte, vous pouvez :
<ul>
    <li>Soit changer de compte Twitter en ré-instanciant un objet tweepy.API à chaque fois que vous avez une exception de type "Rate limit exceeded". N'oubliez pas mettre un time.sleep pour éviter les chaines d'exception trop rapides.</li>
    <li>Soit en collectant les données sur plusieurs notebooks avec vos comptes respectifs. Vous pourrez alors mettre wait_on_rate_limit=True pour éviter les exceptions.</li>
</ul>
Pensez à bien répartir les requêtes par mots clef et dans le temps en utilisant des fenêtres de 1 jour avec les paramètres since et until. Cela permet de paralléliser et de ne pas refaire les mêmes requêtes. Pour rappel, vous ne pourrez pas aller plus loin que 7 jours dans le passé (limite de Twitter).
</div>

## Élagage des données

Conservez uniquement les informations nécessaires à vos analyses (e.g. `id`, `text`, `created_at`, `retweet_count`, `favorite_count`). Vous devez utiliser les attributs de [l'objet Tweet](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object). 

In [6]:
# Élagage : on ne gqrde aue les infos qui nous intéressent
print("New tweets size: " + str(len(tweets)))



tweets_elag = dict()
criteres = ['id', 'text', 'created_at', 'retweet_count', 'favorite_count']
for i in tweets :
    tweets_elag[i]=dict()
    for a in criteres:
        tweets_elag[i][a]= tweets[i][a]

print(tweets_elag)

New tweets size: 120
{1403272444169928704: {'id': 1403272444169928704, 'text': 'Total global sales of new electric car will reach 34% by 2030 and 68% by 2040.', 'created_at': 'Fri Jun 11 08:46:36 +0000 2021', 'retweet_count': 0, 'favorite_count': 0}, 1403272424473432065: {'id': 1403272424473432065, 'text': '@BrightonHoveCC @BHCC_Transport @EBChargingLtd @electricbton There is a charging point in Holland Road #hove it is hardly ever available to electric car drivers as it is made available to non electric cars who park in it, often for complete days of weekends. Why charging points not exclusive to electric vehicles?I\n@BHGreenCllrs @CllrEbel', 'created_at': 'Fri Jun 11 08:46:32 +0000 2021', 'retweet_count': 0, 'favorite_count': 0}, 1403272048051425284: {'id': 1403272048051425284, 'text': 'We’re inviting Expressions of Interest from Fingal based businesses who would like to trial delivery using eCargo bikes. Get access to electric or eCargo bikes &amp; replace your delivery or service t

In [21]:
tweets_text=[]

for dictio in tweets_elag.values():
    tweets_text.append(dictio['text'])
print(tweets_text[0])

Total global sales of new electric car will reach 34% by 2030 and 68% by 2040.


<div class="alert alert-block alert-success">
Avant l'élaguage, vous pouvez aussi <a href="https://stackoverflow.com/questions/27900451/convert-tweepy-status-object-into-json">sérialiser toutes les données des objets Tweepy</a>. Cela peut être utile si, plus tard, vous voulez revenir en arrière, sans devoir refaire les requêtes, pour selectionner un autre champs pour vos analyses (par exemple si vous voulez prendre en compte le nombre de retweets).
</div>

## Sauvegarde des données

Sauvegardez vos tweets avec pickle. L'objet sauvegardé doit être une liste de dictionnaires contenants au moins les champs `id` et `texte`. Exemple : `[{'text': 'Text of the first tweet.', 'id': 10}, {'text': 'Text of the second tweet.', 'id': 11}, ...]`. Vous pouvez bien entendu ajouter vos propres champs (catégorie...).

Votre fichier se nommera `group<X>_<dataname>_projectdata.pickle` avec `X` le numéro de votre groupe et `dataname` le nom de votre projet en enlevant les chevrons. Le fichier est à uploadé sur vos serveurs Jupyter attribués. Il pourra être mis à jour au fur et à mesure.

In [22]:
# Sauvegarde des données en .pickle:
import pickle
file_Name = "group12_electriccars.projectdata.pickle"
# open the file for writing
fileObject = open(file_Name,'wb')

pickle.dump(tweets_text,fileObject) 

fileObject.close()


<div class="alert alert-block alert-success">
Lorsque vous ne connaissez pas un outil ou une notion, le premier réflèxe que vous devez avoir est de <b>comprendre</b> et <b>trouver des exemples</b>. Pour la sauvegarde pickle, vous chercherez par exemple "pickle" sur un moteur de recherche et vous trouverez la <a href="https://docs.python.org/3/library/pickle.html">documentation</a> ou des <a href="https://yasoob.me/2013/08/02/what-is-pickle-in-python">articles expliquant son utilisation</a>. Lorsqu'il s'agit de traitements basiques comme pour la sauvegarde pickle, vous chercherez des exemples à copier en tapant par exemple "How to pickle a list?", vous trouverez alors des <a href="https://stackoverflow.com/questions/25464295/how-to-pickle-a-list/25465148">solutions</a>, souvent sur le site *StackOverflow*.
</div>